In [40]:
# [+] SparkSession 설정
from pyspark.sql import SparkSession

ss = SparkSession.builder.master('local').appName('udf').getOrCreate() # Spark환경 초기화

In [41]:
# 샘플 데이터: 한신대 맛집 및 대표메뉴
hsu_restaurants = [
    ('진현가든', '삼치돌솥밥', '경기 오산시 양산로 340딩', 3.5),
    ('짜장면가', '중화비빔밥, 마파두부밥', '경기 오산시 한신대길 123 경원빌딩', 4.0),
    ('찌개동아리', '제육전골', '경기 오산시 한신대133번길 4', 3.5),
    ('한판삼겹', '항정살', '경기 오산시 양산로410번길 8', 3.5),
    ('화락', '초밥', '경기 오산시 양산로 347 대성빌딩1층', 3.5),
    ('해우리', '고기 덮밥, 해물 라면', '경기 오산시 한신대길 135 1층', 4.5),
    ('행복한콩박사', '맑은순두부', '경기 오산시 양산로398번길 8-11', 3.5)
]


In [42]:
# [+] 스키마 정의
schema = ['restaurant_name', 'speciality', 'address', 'score']

In [43]:
# [+] 데이터프레임 생성
df = ss.createDataFrame(data=hsu_restaurants, schema=schema)

In [44]:
# [+] 데이터프레임 출력
df.show()

+---------------+----------------------+---------------------------------+-----+
|restaurant_name|            speciality|                          address|score|
+---------------+----------------------+---------------------------------+-----+
|       진현가든|            삼치돌솥밥|         경기 오산시 양산로 340딩|  3.5|
|       짜장면가|중화비빔밥, 마파두부밥|경기 오산시 한신대길 123 경원빌딩|  4.0|
|     찌개동아리|              제육전골|      경기 오산시 한신대133번길 4|  3.5|
|       한판삼겹|                항정살|      경기 오산시 양산로410번길 8|  3.5|
|           화락|                  초밥|   경기 오산시 양산로 347 대성...|  3.5|
|         해우리|  고기 덮밥, 해물 라면|     경기 오산시 한신대길 135 1층|  4.5|
|   행복한콩박사|            맑은순두부|   경기 오산시 양산로398번길 8-11|  3.5|
+---------------+----------------------+---------------------------------+-----+



In [45]:
# [+] 데이터프레임 스키마 출력
df.printSchema()

root
 |-- restaurant_name: string (nullable = true)
 |-- speciality: string (nullable = true)
 |-- address: string (nullable = true)
 |-- score: double (nullable = true)



In [46]:
# [+] Temporary View 생성 -> 수정이 안되기 때문에 temporary view만들어서 처리
df.createOrReplaceTempView('restaurants')

### User Defined Function 생성
1. translate(): Google Translation API를 이용하여 한글 식당 이름을 영문으로 번역하는 함수
2. scale_score(): 5점 만점 평점을 100점 스케일로 변환하는 함수

In [47]:
# Google translation 라이브러리 설치
!pip install googletrans==4.0.0-rc1

In [48]:
# [+] Google translator 임포트
from googletrans import Translator

In [49]:
# [+] 번역 테스트
translator = Translator()
result = translator.translate('찌개동아리', src='ko', dest='en') # src:입력문자의 언어 

In [50]:
# [+] 번역 결과 출력
print(result)
# print로 출력하면 Translated라는 객체로 만들어서 return. 잡다한 정보도 출력

Translated(src=ko, dest=en, text=Stew, pronunciation=None, extra_data="{'confiden...")


In [51]:
result.text

'Stew'

In [52]:
# [+] extra_data 출력
result.extra_data
# key-value형태로 출력됨 = dictionary형식
# origin_pronunciation : 영어 발음

{'confidence': None,
 'parts': [<googletrans.models.TranslatedPart at 0x223929be520>],
 'origin_pronunciation': 'jjigaedong-ali',
 'parsed': [['jjigaedong-ali',
   None,
   None,
   [[[0, [[[None, 5]], [True]]]], 5],
   [['찌개동아리', None, None, 5]]],
  [[[None,
     None,
     None,
     None,
     None,
     [['Stew', None, None, None, [['Stew', [5]], ['Jjig', [11]]]]]]],
   'en',
   1,
   'ko',
   ['찌개동아리', 'ko', 'en', True]],
  'ko']}

In [53]:
# [+] 영어 발음(pronunciation) 출력
result.extra_data.get('origin_pronunciation') # dictionary형식에서는 get으로 가져와야함.

'jjigaedong-ali'

In [68]:
from pyspark.sql.functions import udf # annotation방식

In [69]:
# [+] UDF 1: 한글->영문 번역 함수


def translate(text):
    from googletrans import Translator
    translator = Translator() # import하는 준비코드 함수 내에 넣어줘야함
    result = translator.translate(text, src='ko', dest='en')
    return result.extra_data.get('origin_pronunciation')

In [70]:
# [+] UDF 등록 - SQL내에서 사용할 함수(함수명 달라도 되지만 굳이?)
ss.udf.register('translate', translate)

<function __main__.translate(text)>

In [79]:
# [+] SQL문 처리
ss.sql("SELECT restaurant_name, translate(restaurant_name) AS restaurant_eng_name, speciality, address, score FROM restaurants").show()
# name과 speciality사이에 translate넣어주기
# 별칭만드는 키워드 AS 사용헤서 이름 바꾸기

+---------------+--------------------+----------------------+---------------------------------+-----+
|restaurant_name| restaurant_eng_name|            speciality|                          address|score|
+---------------+--------------------+----------------------+---------------------------------+-----+
|       진현가든|      jinhyeongadeun|            삼치돌솥밥|         경기 오산시 양산로 340딩|  3.5|
|       짜장면가|      jjajangmyeonga|중화비빔밥, 마파두부밥|경기 오산시 한신대길 123 경원빌딩|  4.0|
|     찌개동아리|      jjigaedong-ali|              제육전골|      경기 오산시 한신대133번길 4|  3.5|
|       한판삼겹|      hanpansamgyeob|                항정살|      경기 오산시 양산로410번길 8|  3.5|
|           화락|              hwalag|                  초밥|   경기 오산시 양산로 347 대성...|  3.5|
|         해우리|              haeuli|  고기 덮밥, 해물 라면|     경기 오산시 한신대길 135 1층|  4.5|
|   행복한콩박사|haengboghankongbagsa|            맑은순두부|   경기 오산시 양산로398번길 8-11|  3.5|
+---------------+--------------------+----------------------+---------------------------------+-----+



In [80]:
# [+] Annotation 방식으로 UDF 등록하기 -> 함수 정의부에 @udf붙여서 사용하는 것 !!중복해서 사용하면 안됨!!

In [81]:
# [+] UDF 2: score -> 100점 스케일 변환 함수 => 원점수에 2씩 곱하기

def scale_score(score):
    return score * 20

In [82]:
ss.udf.register('scale_score', scale_score)

<function __main__.scale_score(score)>

In [83]:
# [+] SQL문처리 -> 5점만점에서 100점만점 스케일로
ss.sql("SELECT restaurant_name, translate(restaurant_name) AS restaurant_eng_name, \
        speciality, address, scale_score(score) AS scaled_score \
        FROM restaurants").show()

+---------------+--------------------+----------------------+---------------------------------+------------+
|restaurant_name| restaurant_eng_name|            speciality|                          address|scaled_score|
+---------------+--------------------+----------------------+---------------------------------+------------+
|       진현가든|      jinhyeongadeun|            삼치돌솥밥|         경기 오산시 양산로 340딩|        70.0|
|       짜장면가|      jjajangmyeonga|중화비빔밥, 마파두부밥|경기 오산시 한신대길 123 경원빌딩|        80.0|
|     찌개동아리|      jjigaedong-ali|              제육전골|      경기 오산시 한신대133번길 4|        70.0|
|       한판삼겹|      hanpansamgyeob|                항정살|      경기 오산시 양산로410번길 8|        70.0|
|           화락|              hwalag|                  초밥|   경기 오산시 양산로 347 대성...|        70.0|
|         해우리|              haeuli|  고기 덮밥, 해물 라면|     경기 오산시 한신대길 135 1층|        90.0|
|   행복한콩박사|haengboghankongbagsa|            맑은순두부|   경기 오산시 양산로398번길 8-11|        70.0|
+---------------+--------------------+---------